# Diluted Unitary Fitting

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *
from analysis import *
from utils import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [5]:
def optimize_annulus_distance(spectrum_target = None, 
                              d = None, 
                              c_start = 0.5,
                              rank_start = 1,
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001): 
    
    model_list = []
    distance_list = []
    counter = 0
    
    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_unitary(d), 
                                                          c=c_start, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = AnnulusDistance(),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function = AnnulusDistance(), 
                                            verbose=True,N=0),
                            )
        model.train(inputs = None,
                    targets = [spectrum_target],
                    num_iter = num_iter,
                   )

        model_list.append(model)
        distance_list.append(model.logger.loss_train_list[-1])
        
        idx = np.argmin(distance_list)
        if distance_list[-1] > distance_list[idx]:
            counter += 1
        if counter > 4:
            print("Done!")
            break

    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step,
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        U = generate_haar_random(d, rng) 
        distance_best, model_best = optimize_annulus_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## Three Qubits

In [5]:
[channel_list1, channel_list2, channel_list3, channel_list4, channel_list5, channel_list6, channel_list7] = loader(data_path("channel_fullRank_belem_longTrain.p"))

spectrum_target_list1 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list1]
spectrum_target_list2 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list2]
spectrum_target_list3 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list3]
spectrum_target_list4 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list4]
spectrum_target_list5 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list5]
spectrum_target_list6 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list6]
spectrum_target_list7 = [channel_spectrum(channel, keep_unity=False) for channel in channel_list7]

In [4]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list1, 
                                                          d=d, 
                                                          rank_start = 1,
                                                          rank_step = 1,
                                                          num_iter = 2000,
                                                         )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_8layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.26639269787200276 None
0.24281971621747198 None
0.19730583477461208 None
0.15501534293546423 None
0.11784285355313208 None
0.08663346571103148 None
0.060868447156001086 None
0.04435907614063043 None
0.04433281668287777 None
0.04436239396192363 None
0.044334719908165884 None
0.04433354046998839 None
0.04432161704554379 None
0.0443480894883677 None
0.044329318510019665 None
0.04435316001151345 None
0.04433927221222419 None
0.044333789484492785 None
0.04433486518070283 None
0.044403552723975004 None
tf.Tensor(0.04434972719124683, shape=(), dtype=float64)
0.04434434126343461 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.24668894103468048 None
0.21480620374973125 None
0.18370788698612625 None
0.153273739956457 None
0.12402548262459673 None
0.09688616631479197 None
0.07149501941245653 None
0.04736331075227588 None
0.02458847096038777 None
0.017156909402523624 None
0.01715246026962658 None
0.01717258725608535 None
0.017179079968144474 None
0.01717447267632543 None
0.017180122949914174 None
0.017207482139610854 None
0.017213343929584853 None
0.017157175844995362 None
0.0171948959420931 None
0.01720377500600482 None
tf.Tensor(0.017193463676459908, shape=(), dtype=float64)
0.017198457617933205 None


...

  0%|          | 0/2000 [00:00<?, ?it/s]

0.29945884745914886 None
0.2603504455692601 None
0.2219695598117452 None
0.19080107740605134 None
0.16263661471046412 None
0.13595575615250172 None
0.11032832461848226 None
0.08557215135477907 None
0.061866089269453625 None
0.04425581604110401 None
0.04400696527398816 None
0.04402656764107034 None
0.0440311065881768 None
0.04401281920014126 None
0.04401340025244731 None
0.04400423040258977 None
0.04401896234768268 None
0.0440288872104215 None
0.04399053764056517 None
0.04400206905585766 None
tf.Tensor(0.044011149453134155, shape=(), dtype=float64)
0.04399350844377063 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.2610897600456713 None
0.2284617493844322 None
0.1984811320502562 None
0.1708500035747967 None
0.14460640171305228 None
0.11943420833455948 None
0.09530217948635904 None
0.07220954244259187 None
0.050154472575208206 None
0.02914905351848115 None
0.02009630138131774 None
0.020103342896043777 None



KeyboardInterrupt



In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_8layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list2,
                                                          d=d, 
                                                          rank_start = rank_start,
                                                          rank_step = 1,
                                                          num_iter = 2000,
                                                          )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_16layer.model"))

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_16layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list3, 
                                                          d=d, 
                                                          rank_start = rank_start,
                                                          rank_step = 1,
                                                          num_iter = 2000,
                                                         )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_24layer.model"))

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_24layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list4, 
                                                          d=d, 
                                                          rank_start = rank_start,
                                                          rank_step = 1,
                                                          num_iter = 2000,
                                                         )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_32layer.model"))

## Four Qubits

In [3]:
n = 4
d = 2**n

[model1, model2, model3, model4, model5] = loader(data_path("belem_4qubit_repeated.model"))

spectrum1 = channel_spectrum(model1.channel)
spectrum2 = channel_spectrum(model2.channel) 
spectrum3 = channel_spectrum(model3.channel)
spectrum4 = channel_spectrum(model4.channel)
spectrum5 = channel_spectrum(model5.channel)

In [ ]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum1, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 2000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_8layer.model"))

In [ ]:
[distance_best, model_best] = loader(data_path("diluted_fitting_4qubit_8layer.model"))
rank_start = model_best.channel.rank
c_start = model_best.channel.c

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum2, 
                                                      d = d,
                                                      c_start = c_start,
                                                      rank_start = rank_start, 
                                                      rank_step = 1,
                                                      num_iter = 2000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_16layer.model"))

In [6]:
[distance_best, model_best] = loader(data_path("diluted_fitting_4qubit_16layer.model"))
rank_start = model_best.channel.rank
c_start = model_best.channel.c

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      c_start = c_start,
                                                      rank_start = rank_start, 
                                                      rank_step = 1,
                                                      num_iter = 2000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_24layer.model"))

  0%|          | 0/2000 [00:00<?, ?it/s]

0.14024677231318675 None
0.1178863483795487 None
0.09660425285830512 None
0.07644375738522527 None
0.057466756363661206 None
0.03996206404058554 None


In [ ]:
[distance_best, model_best] = loader(data_path("diluted_fitting_4qubit_24layer.model"))
rank_start = model_best.channel.rank
c_start = model_best.channel.c

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum4, 
                                                      d = d,
                                                      c_start = c_start, 
                                                      rank_start = rank_start, 
                                                      rank_step = 1,
                                                      num_iter = 2000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_32layer.model"))